# Part 3: Hands-On - End-to-end Workflow on Titanic Dataset

**🎯 Goal**: We will apply a Supervised Learning workflow on the famous Titanic dataset to predict the survival of passengers.

**🗒️ Scenario**

The Titanic dataset contains information about the passengers, which includes their age, sex, ticket class, and whether they survived the sinking of the Titanic. Refer to the description and data dictionary: https://www.kaggle.com/competitions/titanic/data

**⚡ Task**

## 1. Imports and Data Loading

First, we import the necessary libraries. seaborn is used here specifically to easily load the built-in Titanic dataset. We then inspect the dataframe to understand its structure and check for missing values, which determines our preprocessing strategy.

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
# Load the Titanic dataset from the seaborn library
titanic = sns.load_dataset('titanic')
titanic.shape

(891, 15)

In [3]:
# Display the first 5 rows to inspect data types and example values
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
# Check the distribution of the target variable to see if classes are balanced
titanic.survived.value_counts()

survived
0    549
1    342
Name: count, dtype: int64

In [5]:
# Identify columns with missing values to decide on imputation strategies
# Note: 'age' and 'deck' have significant missing data

titanic.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

## 2. Feature Selection and Splitting

We select a specific subset of features to train our model. We separate the data into the feature matrix (X) and the target vector (y).

In [7]:
# Select features: mixture of numerical (age, fare) and categorical (sex, embaked)
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
target = 'survived'

# Split data into Features (X) and Target (y)
X = titanic[features]
y = titanic[target]


## 3. Building the Pre-Processing Pipeline

This is the core of the workflow. We use Pipeline to chain sequential steps (like imputation and scaling) and ColumnTransformer to apply these different pipelines to specific columns (numerical vs. categorical) simultaneously.

- **Numerical Data**: We fill missing values with the median and scale data to unit variance.

- **Categorical Data**: We fill missing values with the most frequent value and convert text categories into binary vectors (One-Hot Encoding).

`sklearn`'s pipeline is a tool that allows us to assemble several steps together. It sequentially applies a list of transforms and a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit.

![Pipeline Overview](../assets/titanic-data-pipeline.png)

In [8]:
# 1. Define pipeline for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Fill missing values with median
    ('scaler', StandardScaler())                   # Standardize features (mean=0, variance=1)
])

# 2. Define pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Fill missing with mode
    ('onehot', OneHotEncoder(handle_unknown='ignore'))    # Convert categories to binary vectors
])

# 3. Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        # Apply numerical pipeline to specific numeric columns
        ('num', numerical_transformer, ['age', 'sibsp', 'parch', 'fare']),
        # Apply categorical pipeline to specific categorical columns
        ('cat', categorical_transformer, ['pclass', 'sex', 'embarked'])
    ])

# 4. Create the full end-to-end pipeline including the model
# This ensures raw data flows through preprocessing directly into the model
model = LogisticRegression()
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

## 4. Training and Evaluation

Finally, we split the data into training and testing sets. We fit the entire pipeline on the training data and evaluate its performance on the unseen test data using various classification metrics.

In [9]:
# Split data: 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

# Calculate classification metrics
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

# Calculate ROC curve and AUC score
# Note: We use predict_proba for ROC/AUC to get probability scores instead of class labels
fpr, tpr, thresholds = roc_curve(y_test, pipeline.predict_proba(X_test)[:,1])
roc_auc = auc(fpr, tpr)

# Output results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f"AUC: {roc_auc:.2f}")

Accuracy: 0.80
Precision: 0.75
Recall: 0.74
F1 Score: 0.74
AUC: 0.87


## Exercise (Self-Study)

> Now, recreate the workflow but use min-max scaling for numerical features and KNN classifier for model.

We redefine the numerical pipeline to use `MinMaxScaler` instead of `StandardScaler`. We then integrate this into a new preprocessor and combine it with a KNeighborsClassifier.

In [10]:
from sklearn.neighbors import KNeighborsClassifier

# 1. Redefine numerical pipeline with Min-Max Scaling
# Min-Max scaling scales data to a fixed range [0, 1], which preserves the shape of the original distribution
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Fill missing values
    ('scaler', MinMaxScaler())                     # Scale to range [0, 1]
])

# 2. Update the ColumnTransformer
# We reuse the 'categorical_transformer' defined in the previous section (Imputer + OneHotEncoder)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, ['age', 'sibsp', 'parch', 'fare']),
        ('cat', categorical_transformer, ['pclass', 'sex', 'embarked'])
    ])

# 3. Define the new model: K-Nearest Neighbors
model = KNeighborsClassifier()

# 4. Create the new pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])


**Training and Evaluation**

We fit this new pipeline to the same training data used previously and evaluate its performance. This allows for a direct comparison between the Logistic Regression (StandardScaler) approach and this KNN (MinMaxScaler) approach.

In [11]:

# Train the KNN pipeline
pipeline.fit(X_train, y_train)

# Generate predictions
preds = pipeline.predict(X_test)

# Calculate classification metrics
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

# Calculate ROC/AUC
# Note: KNN supports predict_proba, which allows us to calculate AUC just like Logistic Regression
fpr, tpr, thresholds = roc_curve(y_test, pipeline.predict_proba(X_test)[:,1])
roc_auc = auc(fpr, tpr)

In [12]:
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f"AUC: {roc_auc:.2f}")

Accuracy: 0.81
Precision: 0.79
Recall: 0.70
F1 Score: 0.74
AUC: 0.85
